In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings; warnings.simplefilter('ignore')
from scipy import stats
from ast import literal_eval
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import wordnet
from surprise import Reader, Dataset, SVD
from surprise.model_selection import cross_validate
import sys, csv ,operator

### OLD DATA

In [22]:
#USER-BOOK RATING
ratings=pd.read_csv('reference/CustomData/ratings.csv')
ratings

,user_id,book_id,rating
0,1,258,5
1,2,260,5
2,2,26,4
3,2,315,3
4,2,33,4
...,...,...,...
3133539,49925,330,5
3133540,49925,510,5
3133541,49925,528,4
3133542,49925,722,4


In [58]:
###Average rating of book
avgRatings=pd.read_csv('reference/CustomData/AverageRatings.csv')
avgRatings

,book_id,rating
0,1,4.279707
1,2,4.351350
2,3,3.214341
3,4,4.329369
4,5,3.772224
...,...,...
9995,9996,4.014184
9996,9997,4.451613
9997,9998,4.323529
9998,9999,3.707692


In [57]:
##Counts of book ratings 
ratingCounts=pd.read_csv('reference/CustomData/RatingsCount.csv')
ratingCounts

,book_id,rating
0,1,22806
1,2,21850
2,3,16931
3,4,19088
4,5,16604
...,...,...
9995,9996,141
9996,9997,93
9997,9998,102
9998,9999,130


In [67]:
###BOOKS INFO
finalData=pd.read_csv('reference/CustomData/FinalData.csv')
finalData

,book_id,authors,title,Genres
0,1,Suzanne Collins,"The Hunger Games (The Hunger Games, #1)",SciFi;Drama
1,2,"J.K. Rowling, Mary GrandPré",Harry Potter and the Sorcerer's Stone (Harry P...,Fantasy;Young-Age
2,3,Stephenie Meyer,"Twilight (Twilight, #1)",Fantasy
3,4,Harper Lee,To Kill a Mockingbird,Self-Help;Drama
4,5,F. Scott Fitzgerald,The Great Gatsby,Drama
...,...,...,...,...
994,995,Michelle Hodkin,"The Unbecoming of Mara Dyer (Mara Dyer, #1)",Fiction
995,996,Janet Evanovich,"Three to Get Deadly (Stephanie Plum, #3)",Crime
996,997,Emmuska Orczy,The Scarlet Pimpernel,History;Fiction
997,998,"Jon Stone, Michael J. Smollin",The Monster at the End of this Book,Fiction;Kids


In [69]:
finalRatings = pd.read_csv('reference/CustomData/FinalRatings.csv')
finalRatings

,user_id,book_id,rating
0,1,258,5
1,2,260,5
2,2,26,4
3,2,315,3
4,2,33,4
...,...,...,...
3133539,49925,330,5
3133540,49925,510,5
3133541,49925,528,4
3133542,49925,722,4


### NEW DATASET

In [44]:
book_data=pd.read_csv('dataset/BX-Books.csv', sep=';', error_bad_lines=False,encoding='latin-1')
user_data=pd.read_csv('dataset/BX-Users.csv', sep=';', error_bad_lines=False,encoding='latin-1')
book_ratings=pd.read_csv('dataset/BX-Book-Ratings.csv', sep=';', error_bad_lines=False,encoding='latin-1')

b'Skipping line 6452: expected 8 fields, saw 9\nSkipping line 43667: expected 8 fields, saw 10\nSkipping line 51751: expected 8 fields, saw 9\n'
b'Skipping line 92038: expected 8 fields, saw 9\nSkipping line 104319: expected 8 fields, saw 9\nSkipping line 121768: expected 8 fields, saw 9\n'
b'Skipping line 144058: expected 8 fields, saw 9\nSkipping line 150789: expected 8 fields, saw 9\nSkipping line 157128: expected 8 fields, saw 9\nSkipping line 180189: expected 8 fields, saw 9\nSkipping line 185738: expected 8 fields, saw 9\n'
b'Skipping line 209388: expected 8 fields, saw 9\nSkipping line 220626: expected 8 fields, saw 9\nSkipping line 227933: expected 8 fields, saw 11\nSkipping line 228957: expected 8 fields, saw 10\nSkipping line 245933: expected 8 fields, saw 9\nSkipping line 251296: expected 8 fields, saw 9\nSkipping line 259941: expected 8 fields, saw 9\nSkipping line 261529: expected 8 fields, saw 9\n'


In [45]:
book_ratings

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6
...,...,...,...
1149775,276704,1563526298,9
1149776,276706,0679447156,0
1149777,276709,0515107662,10
1149778,276721,0590442449,10


In [46]:
book_data

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...
...,...,...,...,...,...,...,...,...
271355,0440400988,There's a Bat in Bunk Five,Paula Danziger,1988,Random House Childrens Pub (Mm),http://images.amazon.com/images/P/0440400988.0...,http://images.amazon.com/images/P/0440400988.0...,http://images.amazon.com/images/P/0440400988.0...
271356,0525447644,From One to One Hundred,Teri Sloat,1991,Dutton Books,http://images.amazon.com/images/P/0525447644.0...,http://images.amazon.com/images/P/0525447644.0...,http://images.amazon.com/images/P/0525447644.0...
271357,006008667X,Lily Dale : The True Story of the Town that Ta...,Christine Wicker,2004,HarperSanFrancisco,http://images.amazon.com/images/P/006008667X.0...,http://images.amazon.com/images/P/006008667X.0...,http://images.amazon.com/images/P/006008667X.0...
271358,0192126040,Republic (World's Classics),Plato,1996,Oxford University Press,http://images.amazon.com/images/P/0192126040.0...,http://images.amazon.com/images/P/0192126040.0...,http://images.amazon.com/images/P/0192126040.0...


In [ ]:
books2=book_data[['ISBN','Book-Title','Book-Author','Year-Of-Publication','Publisher']]
books2.to_csv('dataset/processed/FinalData.csv',sep=',')

In [47]:
user_data

,User-ID,Location,Age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0
2,3,"moscow, yukon territory, russia",NaN
3,4,"porto, v.n.gaia, portugal",17.0
4,5,"farnborough, hants, united kingdom",NaN
...,...,...,...
278853,278854,"portland, oregon, usa",NaN
278854,278855,"tacoma, washington, united kingdom",50.0
278855,278856,"brampton, ontario, canada",NaN
278856,278857,"knoxville, tennessee, usa",NaN


### PROCESSING DATASETS

In [8]:
def prepocess_books_ratings():
    res=[]
    cnt=0
    with open('dataset/BX-Book-Ratings.csv','r',encoding="latin-1") as in1,open('dataset/BX-Books.csv','r',encoding="latin-1")as in2,open('dataset/processed/FinalRatings.csv','w')as out:
        for x in in2:
            x=x.split(';')
            res.append(x[0])
        res=res[0:1000]
        for x in in1:
            x=x.split(';')
            if x[1] in res:
                r= x[0]+','+x[1]+','+x[2]
                out.write(r)
                cnt+=1
    print(cnt)
    print(len(res))

49968
1000


In [111]:
def preprocess_average_stats_ratings():
    df = pd.read_csv('dataset/BX-Book-Ratings.csv', sep=';', error_bad_lines=False,encoding='latin-1')
    dic = df.groupby('ISBN')[['Book-Rating']].mean()
    dic1 = df.groupby('ISBN')[['Book-Rating']].count()
    file = 'dataset/processed/AverageCount.csv'
    dic.to_csv(file, sep=',')
    file1 = 'dataset/processed/RatingsCount.csv'
    dic1.to_csv(file1, sep=',')

In [9]:
avgRatings=pd.read_csv('dataset/processed/AverageRatings.csv')
avgRatings

,book_id,rating
0,0330299891,3.0
1,0375404120,1.5
2,0586045007,0.0
3,9022906116,3.5
4,9032803328,0.0
...,...,...
340551,cn113107,0.0
340552,ooo7156103,7.0
340553,§423350229,0.0
340554,´3499128624,8.0


In [104]:
avgRatings=avgRatings.rename(columns={"ISBN": "book_id", "Book-Rating": "rating"})
avgRatings.to_csv('dataset/processed/AverageRatings.csv',sep=',',index=False)

In [10]:
ratingCounts=pd.read_csv('dataset/processed/RatingsCount.csv')
ratingCounts

,book_id,rating
0,0330299891,2
1,0375404120,2
2,0586045007,1
3,9022906116,2
4,9032803328,1
...,...,...
340551,cn113107,1
340552,ooo7156103,1
340553,§423350229,1
340554,´3499128624,1


In [113]:
ratingCounts=ratingCounts.rename(columns={"ISBN": "book_id", "Book-Rating": "rating"})
ratingCounts.to_csv('dataset/processed/RatingsCount.csv',sep=',',index=False)

In [13]:
finalRatings = pd.read_csv('dataset/processed/FinalRatings.csv')
finalRatings

,user_id,book_id,rating
0,276746,0449006522,0
1,276746,055356451X,0
2,276755,0451166892,5
3,276796,0330332775,5
4,276847,0446364193,0
...,...,...,...
49962,276681,0399144463,8
49963,276681,0446677450,8
49964,276688,006101351X,0
49965,276688,068484267X,0


In [12]:
finalRatings=finalRatings.rename(columns={"User-ID":"user_id","ISBN": "book_id", "Book-Rating": "rating"})
finalRatings.to_csv('dataset/processed/FinalRatings.csv',sep=',',index=False)

In [17]:
finalData=pd.read_csv('dataset/processed/FinalData.csv')
finalData

,book_id,title,authors,year,publisher
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company
...,...,...,...,...,...
995,0425166619,Toxin,Robin Cook,1999,Berkley Publishing Group
996,061312975X,This Present Darkness,Frank E. Peretti,1999,Sagebrush Bound
997,0765341972,The Mothman Prophecies,John A. Keel,2002,Tor Books
998,0590514776,Meet the Stars of Buffy the Vampire Slayer,Stefanie Scott,1998,Scholastic


In [16]:
finalData=finalData.rename(columns={"Book-Title":"title","ISBN": "book_id", "Book-Author": "authors","Year-Of-Publication":"year","Publisher":"publisher"})
finalData=finalData[0:1000]
finalData.to_csv('dataset/processed/FinalData.csv',sep=',',index=False)